In [5]:
#basic libraries
import pandas as pd 
import numpy as np 

#for preprocessing
from sklearn.preprocessing import MinMaxScaler

#for splitting dataset later 
from sklearn.model_selection import train_test_split



In [6]:
col_names = [
    "duration","protocol_type","service","flag","src_bytes","dst_bytes",
    "land","wrong_fragment","urgent","hot","num_failed_logins","logged_in",
    "num_compromised","root_shell","su_attempted","num_root","num_file_creations",
    "num_shells","num_access_files","num_outbound_cmds","is_host_login","is_guest_login",
    "count","srv_count","serror_rate","srv_serror_rate","rerror_rate","srv_rerror_rate",
    "same_srv_rate","diff_srv_rate","srv_diff_host_rate","dst_host_count",
    "dst_host_srv_count","dst_host_same_srv_rate","dst_host_diff_srv_rate",
    "dst_host_same_src_port_rate","dst_host_srv_diff_host_rate",
    "dst_host_serror_rate","dst_host_srv_serror_rate","dst_host_rerror_rate",
    "dst_host_srv_rerror_rate","label","difficulty"
]

In [7]:
train_df= pd.read_csv("C:\\Users\\Dell\\OneDrive\\Desktop\\Projects\\X-IDS\\nsl-kdd\\KDDTrain+.txt", names=col_names)
test_df= pd.read_csv("C:\\Users\\Dell\\OneDrive\\Desktop\\Projects\\X-IDS\\nsl-kdd\\KDDTest+.txt", names=col_names)
print(train_df.head())


   duration protocol_type   service flag  src_bytes  dst_bytes  land  \
0         0           tcp  ftp_data   SF        491          0     0   
1         0           udp     other   SF        146          0     0   
2         0           tcp   private   S0          0          0     0   
3         0           tcp      http   SF        232       8153     0   
4         0           tcp      http   SF        199        420     0   

   wrong_fragment  urgent  hot  ...  dst_host_same_srv_rate  \
0               0       0    0  ...                    0.17   
1               0       0    0  ...                    0.00   
2               0       0    0  ...                    0.10   
3               0       0    0  ...                    1.00   
4               0       0    0  ...                    1.00   

   dst_host_diff_srv_rate  dst_host_same_src_port_rate  \
0                    0.03                         0.17   
1                    0.60                         0.88   
2             

In [8]:
print(test_df.head())

   duration protocol_type   service  flag  src_bytes  dst_bytes  land  \
0         0           tcp   private   REJ          0          0     0   
1         0           tcp   private   REJ          0          0     0   
2         2           tcp  ftp_data    SF      12983          0     0   
3         0          icmp     eco_i    SF         20          0     0   
4         1           tcp    telnet  RSTO          0         15     0   

   wrong_fragment  urgent  hot  ...  dst_host_same_srv_rate  \
0               0       0    0  ...                    0.04   
1               0       0    0  ...                    0.00   
2               0       0    0  ...                    0.61   
3               0       0    0  ...                    1.00   
4               0       0    0  ...                    0.31   

   dst_host_diff_srv_rate  dst_host_same_src_port_rate  \
0                    0.06                         0.00   
1                    0.06                         0.00   
2       

In [9]:
# Drop 'difficulty' if present
if 'difficulty' in train_df.columns:
    train_df = train_df.drop(['difficulty'], axis=1)
if 'difficulty' in test_df.columns:
    test_df = test_df.drop(['difficulty'], axis=1)


In [10]:
from sklearn.preprocessing import LabelEncoder

# Encode categorical columns
categorical_cols = ['protocol_type', 'service', 'flag']
for col in categorical_cols:
    le = LabelEncoder()
    train_df[col] = le.fit_transform(train_df[col])
    test_df[col] = le.transform(test_df[col])  # Use same encoder to avoid mismatch


In [11]:
# Mapping dictionary
attack_map = {
    'normal': 'normal',

    # DoS
    'neptune':'dos','smurf':'dos','back':'dos','teardrop':'dos','land':'dos','pod':'dos',
    'apache2':'dos','udpstorm':'dos','processtable':'dos','worm':'dos',

    # Probe
    'satan':'probe','ipsweep':'probe','nmap':'probe','portsweep':'probe','mscan':'probe','saint':'probe',

    # R2L
    'guess_passwd':'r2l','ftp_write':'r2l','imap':'r2l','phf':'r2l','multihop':'r2l',
    'warezmaster':'r2l','warezclient':'r2l','spy':'r2l','xlock':'r2l','xsnoop':'r2l',
    'snmpguess':'r2l','snmpgetattack':'r2l','httptunnel':'r2l','sendmail':'r2l','named':'r2l',

    # U2R
    'buffer_overflow':'u2r','loadmodule':'u2r','rootkit':'u2r','perl':'u2r','sqlattack':'u2r',
    'xterm':'u2r','ps':'u2r'
}

# Apply mapping
train_df['attack_cat'] = train_df['label'].map(attack_map)
test_df['attack_cat']  = test_df['label'].map(attack_map)

# Check distribution
print(train_df['attack_cat'].value_counts())
print(test_df['attack_cat'].value_counts())


attack_cat
normal    67343
dos       45927
probe     11656
r2l         995
u2r          52
Name: count, dtype: int64
attack_cat
normal    9711
dos       7167
r2l       2885
probe     2421
u2r         67
Name: count, dtype: int64


In [12]:
print('attack_cat' in train_df.columns)
print('attack_cat' in test_df.columns)

True
True


In [13]:
print(train_df.columns.tolist())


['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'label', 'attack_cat']


In [14]:
#scaling numerical features 
from sklearn.preprocessing import MinMaxScaler

# Drop label columns before scaling
X_train = train_df.drop(['label', 'attack_cat'], axis=1)
X_test  = test_df.drop(['label', 'attack_cat'], axis=1)

In [15]:
# Save target variables
y_train = train_df['attack_cat']
y_test  = test_df['attack_cat']

# Scale features
scaler = MinMaxScaler()
X_train[X_train.columns] = scaler.fit_transform(X_train)
X_test[X_test.columns]   = scaler.transform(X_test)

print("Train features shape:", X_train.shape)
print("Test features shape:", X_test.shape)

Train features shape: (125973, 41)
Test features shape: (22544, 41)


In [16]:
# Drop rows where attack_cat is NaN
train_df = train_df.dropna(subset=['attack_cat'])
test_df  = test_df.dropna(subset=['attack_cat'])
print(train_df)
print(test_df)


        duration  protocol_type  service  flag  src_bytes  dst_bytes  land  \
0              0              1       20     9        491          0     0   
1              0              2       44     9        146          0     0   
2              0              1       49     5          0          0     0   
3              0              1       24     9        232       8153     0   
4              0              1       24     9        199        420     0   
...          ...            ...      ...   ...        ...        ...   ...   
125968         0              1       49     5          0          0     0   
125969         8              2       49     9        105        145     0   
125970         0              1       54     9       2231        384     0   
125971         0              1       30     5          0          0     0   
125972         0              1       20     9        151          0     0   

        wrong_fragment  urgent  hot  ...  dst_host_same_srv_rat

In [17]:
# Drop rows with NaN in attack_cat
test_df = test_df.dropna(subset=['attack_cat'])

# Reassign target variables
y_train = train_df['attack_cat']
y_test  = test_df['attack_cat']



In [18]:
print("Missing attack_cat in train:", train_df['attack_cat'].isna().sum())
print("Missing attack_cat in test:", test_df['attack_cat'].isna().sum())


Missing attack_cat in train: 0
Missing attack_cat in test: 0


In [19]:
print(set(y_train))
print(set(y_test))


{'normal', 'u2r', 'r2l', 'dos', 'probe'}
{'normal', 'u2r', 'r2l', 'dos', 'probe'}


In [20]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(list(y_train) + list(y_test))  # Fit on all labels to avoid unseen label error
y_train_encoded = le.transform(y_train)
y_test_encoded  = le.transform(y_test)

print(dict(zip(le.classes_, le.transform(le.classes_))))



{np.str_('dos'): np.int64(0), np.str_('normal'): np.int64(1), np.str_('probe'): np.int64(2), np.str_('r2l'): np.int64(3), np.str_('u2r'): np.int64(4)}


In [21]:
# After dropping NaNs from test_df
X_test = test_df.drop(['label', 'attack_cat'], axis=1)

# Re-scale X_test
X_test[X_test.columns] = scaler.transform(X_test)
print(X_test)


       duration  protocol_type   service  flag     src_bytes     dst_bytes  \
0      0.000000            0.5  0.710145   0.1  0.000000e+00  0.000000e+00   
1      0.000000            0.5  0.710145   0.1  0.000000e+00  0.000000e+00   
2      0.000047            0.5  0.289855   0.9  9.408217e-06  0.000000e+00   
3      0.000000            0.0  0.202899   0.9  1.449313e-08  0.000000e+00   
4      0.000023            0.5  0.869565   0.2  0.000000e+00  1.145093e-08   
...         ...            ...       ...   ...           ...           ...   
22539  0.000000            0.5  0.782609   0.9  5.753774e-07  2.542106e-07   
22540  0.000000            0.5  0.347826   0.9  2.297162e-07  7.160648e-07   
22541  0.000000            0.5  0.347826   0.9  3.952277e-05  6.346868e-06   
22542  0.000000            1.0  0.173913   0.9  3.043558e-08  3.206260e-08   
22543  0.000000            0.5  0.826087   0.1  0.000000e+00  0.000000e+00   

       land  wrong_fragment  urgent       hot  ...  dst_host_co

In [22]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Instantiate the models
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')


# Train Random Forest
rf_model.fit(X_train, y_train_encoded)
rf_preds = rf_model.predict(X_test)

# Train XGBoost
xgb_model.fit(X_train, y_train_encoded)
xgb_preds = xgb_model.predict(X_test)

# Evaluate both models
from sklearn.metrics import classification_report

print("🔍 Random Forest Performance:\n", classification_report(y_test_encoded, rf_preds, target_names=le.classes_))
print("🔍 XGBoost Performance:\n", classification_report(y_test_encoded, xgb_preds, target_names=le.classes_))




C:\Users\Dell\AppData\Roaming\Python\Python311\site-packages\xgboost\training.py:183: UserWarning: [20:20:42] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


🔍 Random Forest Performance:
               precision    recall  f1-score   support

         dos       0.96      0.83      0.89      7167
      normal       0.66      0.97      0.79      9711
       probe       0.86      0.64      0.73      2421
         r2l       1.00      0.00      0.01      2885
         u2r       0.25      0.01      0.03        67

    accuracy                           0.76     22251
   macro avg       0.75      0.49      0.49     22251
weighted avg       0.82      0.76      0.71     22251

🔍 XGBoost Performance:
               precision    recall  f1-score   support

         dos       0.96      0.86      0.91      7167
      normal       0.69      0.97      0.80      9711
       probe       0.85      0.66      0.74      2421
         r2l       0.98      0.07      0.13      2885
         u2r       0.55      0.09      0.15        67

    accuracy                           0.78     22251
   macro avg       0.80      0.53      0.55     22251
weighted avg       0.83

In [23]:
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

# Step 1: Define SMOTE and undersampler
smote = SMOTE(random_state=42)
undersample = RandomUnderSampler(random_state=42)

# Step 2: Create pipeline
pipeline = Pipeline([
    ('smote', smote),
    ('undersample', undersample)
])

# Step 3: Apply to training data
X_train_balanced, y_train_balanced = pipeline.fit_resample(X_train, y_train_encoded)

print(X_train_balanced, y_train_balanced)


ImportError: cannot import name 'EstimatorCheckFailedWarning' from 'sklearn.exceptions' (C:\Users\Dell\AppData\Roaming\Python\Python311\site-packages\sklearn\exceptions.py)

In [ ]:
rf_model.fit(X_train_balanced, y_train_balanced)
xgb_model.fit(X_train_balanced, y_train_balanced)


In [ ]:
rf_preds = rf_model.predict(X_test)
xgb_preds = xgb_model.predict(X_test)


In [24]:
# ==========================================
# 🧠 SHAP Explainability Section
# ==========================================
import shap
import matplotlib.pyplot as plt

# Initialize SHAP explainers
explainer_rf = shap.TreeExplainer(rf_model)
shap_values_rf = explainer_rf.shap_values(X_test)

# --- SHAP summary plot (Feature importance)
plt.title("Random Forest Feature Importance (SHAP Values)")
shap.summary_plot(shap_values_rf, X_test, plot_type="bar")

# --- SHAP force plot for a specific sample
sample_index = 10  # you can change this
shap.initjs()
shap.force_plot(
    explainer_rf.expected_value[0],
    shap_values_rf[0][sample_index],
    X_test.iloc[sample_index],
)



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\Dell\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\Dell\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\Dell\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import